In [ ]:
%gui qt

In [ ]:
from importlib import reload

import numpy as np

import pyqtgraph as pg
from pyqtgraph.flowchart import Flowchart, Node as pgNode
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.flowchart import library as fclib

from plottr.data import datadict; reload(datadict)
from plottr.data.datadict import togrid, DataDict, GridDataDict

In [ ]:
def testdata_1d(nvals=11):
    x = np.linspace(0, 10, nvals)
    y = np.cos(x)
    z = np.cos(x)**2
    d = DataDict(
        x = {'values' : x},
        y = {'values' : y, 'axes' : ['x']},
        z = {'values' : z, 'axes' : ['x']},
    )
    return d

def testdata_3d(nx=3, ny=3, nz=3):
    x = np.linspace(0, 10, nx)
    y = np.linspace(-5, 5, ny)
    z = np.arange(nz)
    xx, yy, zz = np.meshgrid(x, y, z, indexing='ij')
    dd = np.cos(xx) * np.sin(yy) + np.random.rand(*zz.shape)
    dd2 = np.sin(xx) * np.cos(yy) + np.random.rand(*zz.shape)
    d = DataDict(
        x = dict(values=xx.reshape(-1)),
        y = dict(values=yy.reshape(-1)),
        z = dict(values=zz.reshape(-1)),
        data = dict(values=dd.reshape(-1), axes=['x', 'y', 'z']),
        more_data = dict(values=dd2.reshape(-1), axes=['x', 'y', 'z']),
        different_data = dict(values=dd2.T.reshape(-1), axes=['z', 'y', 'x'])
    )
    return d

In [ ]:
from plottr import node; reload(node)
from plottr.node import Node, DataSelector

Node.raiseExceptions = True
    
nodelib = fclib.NodeLibrary()
nodelib.addNodeType(Node, [('Basic')])
nodelib.addNodeType(DataSelector, [('Basic')])

fc = Flowchart(terminals={
    'dataIn': {'io': 'in'},
    'dataOut': {'io': 'out'}
})
fc.library = nodelib
fc.setInput(dataIn=testdata_3d())

selector = fc.createNode('DataSelector')

fc.connectTerminals(fc['dataIn'], selector['dataIn'])
fc.connectTerminals(selector['dataOut'], fc['dataOut'])

In [ ]:
fc.inputValues(), fc.output()

In [ ]:
selector.signalUpdate = True
selector.selectedData = ['data', 'more_data']

In [ ]:
fc.inputValues(), fc.output()